In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

class DecisionTree:

    def __init__(self, max_depth=None, min_samples_split=None, min_samples_leaf=None, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features

    def gini(self, labels):
        unique, counts = np.unique(labels, return_counts=True)
        proportions = counts / np.sum(counts)
        return 1 - np.sum(proportions ** 2)

    def split(self, X, y, feature_idx, threshold):
        left_idxs = np.where(X[:, feature_idx] <= threshold)[0]
        right_idxs = np.where(X[:, feature_idx] > threshold)[0]
        left_X, left_y = X[left_idxs], y[left_idxs]
        right_X, right_y = X[right_idxs], y[right_idxs]
        return left_X, left_y, right_X, right_y

    def best_split(self, X, y):
        best_feature_idx, best_threshold, best_gini = None, None, np.inf
        features = np.arange(X.shape[1])
        if self.max_features is not None and self.max_features <= X.shape[1]:
            features = np.random.choice(features, size=int(self.max_features), replace=False)
        for feature_idx in features:
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_X, left_y, right_X, right_y = self.split(X, y, feature_idx, threshold)
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                if self.min_samples_leaf is not None:
                    if len(left_y) < self.min_samples_leaf or len(right_y) < self.min_samples_leaf:
                        continue
                left_gini = self.gini(left_y)
                right_gini = self.gini(right_y)
                weighted_gini = (len(left_y) / len(y)) * left_gini + (len(right_y) / len(y)) * right_gini
                if weighted_gini < best_gini:
                    best_feature_idx = feature_idx
                    best_threshold = threshold
                    best_gini = weighted_gini
        return best_feature_idx, best_threshold

    def fit(self, X, y, depth=0):
        if depth == self.max_depth or len(np.unique(y)) == 1:
            return np.bincount(y).argmax()
        if self.min_samples_split is not None and len(y) < self.min_samples_split:
            return np.bincount(y).argmax()
        if self.min_samples_leaf is not None and len(y) < 2 * self.min_samples_leaf:
            return np.bincount(y).argmax()
        feature_idx, threshold = self.best_split(X, y)
        if feature_idx is None:
            return np.bincount(y).argmax()
        left_X, left_y, right_X, right_y = self.split(X, y, feature_idx, threshold)
        left_subtree = self.fit(left_X, left_y, depth + 1)
        right_subtree = self.fit(right_X, right_y, depth + 1)
        return {'feature_idx': feature_idx, 'threshold': threshold, 'left': left_subtree, 'right': right_subtree}

    def predict(self, X, tree):
        if isinstance(tree, np.int64):
            return tree
        feature_idx, threshold = tree['feature_idx'], tree['threshold']
        if X[feature_idx] <= threshold:
            if isinstance(tree['left'], np.int64):
                return tree['left']
            else:
                return self.predict(X, tree['left'])
        else:
            if isinstance(tree['right'], np.int64):
                return tree['right']
            else:
                return self.predict(X, tree['right'])


# Train Test Split
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)

    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)

    # Shuffle the data and target arrays
    shuffled_idxs = np.random.permutation(n_samples)
    X_shuffled = X[shuffled_idxs]
    y_shuffled = y[shuffled_idxs]

    # Split the data and target arrays into training and test sets
    X_train, X_test = X_shuffled[:-n_test], X_shuffled[-n_test:]
    y_train, y_test = y_shuffled[:-n_test], y_shuffled[-n_test:]

    return X_train, X_test, y_train, y_test


# Label Encoding
def encode_data(data):
    label_encoder = LabelEncoder()
    for column in data.columns:
         # check if the column contains boolean data
        if data[column].dtype == bool:
            data[column] = data[column].astype(int)
        # check if the column contains categorical data
        elif data[column].dtype == object or pd.api.types.is_string_dtype(data[column]):
            data[column] = label_encoder.fit_transform(data[column])
        # check if the column contains numeric data
        elif pd.api.types.is_numeric_dtype(data[column]):
            pass  # no encoding needed for numeric data
        else:
            raise ValueError(f"Unsupported data type in column {column}: {data[column].dtype}")
    return data


In [2]:
# file = pd.read_csv("weather.csv")
# print(file)

# data = encode_data(file)
# print(data)

# X = data.drop(['play'], axis = 1)
# y = data['play']
# np_X = X.to_numpy()
# np_y = y.to_numpy()
# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(np_X, np_y, test_size=0.2, random_state=42)

# # Fit the decision tree on the training data
# dt = DecisionTree()
# tree = dt.fit(X_train, y_train)

# # Make predictions on the testing data
# y_pred = [dt.predict(X, tree) for X in X_test]

# # Evaluate the accuracy of the model
# accuracy = sum(y_pred == y_test) / len(y_test)
# print(f"Accuracy: {accuracy}")

# print(tree)

## Decision Tree Genetic Algorithm Optimization




In [3]:
import random

def fitness(individual):
    clf = DecisionTree(
        max_depth=individual[0],
        min_samples_split=individual[1],
        min_samples_leaf=individual[2],
        max_features=individual[3]
    )
    tree = clf.fit(X_train, y_train)
    y_pred = [clf.predict(X, tree) for X in X_test]
    accuracy = sum(y_pred == y_test) / len(y_test)
    return accuracy

def create_individual():
    individual = []
    individual.append(random.randint(1, 20))
    individual.append(random.uniform(0.01, 0.1))
    individual.append(random.uniform(0.01, 0.1))
    individual.append(random.randint(5, 10)*0.1)
    return individual

def generate_population(population_size):
  population = [create_individual() for i in range(population_size)]
  return population

def rank(fitness_values):
    n = len(fitness_values)
    ranks = [0] * n
    sorted_indices = sorted(range(n), key=lambda x: fitness_values[x], reverse=True)
    for i, idx in enumerate(sorted_indices):
        ranks[idx] = i + 1
    return ranks

def selection_probabilities_rank(ranks):
    population_size = len(ranks)

    # Calculate the sum of the ranks
    rank_sum = sum(range(1, population_size + 1))

    # Calculate the selection probability for each individual based on its rank
    selection_probs = [(population_size - rank + 1) / rank_sum for rank in ranks]

    return selection_probs


def select_parents_rank(population, selection_probabilities, num_parents):
    parents = []
    for i in range(num_parents):
        selected_index = np.random.choice(len(population), p=selection_probabilities)
        parents.append(population[selected_index])
    return parents

def crossover(parent1, parent2, crossover_rate):
    child1 = parent1[:]
    child2 = parent2[:]

    for i in range(1, len(parent1)):
        if random.random() < crossover_rate:
            child1[i], child2[i] = child2[i], child1[i]

    return child1, child2

def mutate(individual, mutation_rate):
    for i in range(1, len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]*random.random()
    return individual


def genetic_algorithm(population_size, generations, crossover_rate, mutation_rate):
    # Generate initial population
    population = generate_population(population_size)
    best_individual_ = create_individual()
    best_fitness = fitness(best_individual_)

    for i in range(generations):
        # Calculate fitness for each individual
        fitness_values = [fitness(individual) for individual in population]

        best_individual = max(population, key=fitness)
        curr_best_fitness = fitness(best_individual)
        if curr_best_fitness > best_fitness:
            best_individual_ = best_individual
            best_fitness = curr_best_fitness

        print("Iteration ", i)
        print("curr_best: ", curr_best_fitness, "  best_so_far: ", best_fitness)
        print(best_individual)
        print(best_individual_)
        print()
        print()

        # Implement elitism
        elite_size = int(0.1 * population_size) # top 10% of population
        sorted_population = [x for _, x in sorted(zip(fitness_values, population), reverse=True)]
        elite = sorted_population[:elite_size]

        # Select parents for crossover
        # Implement rank-based selection
        ranks = rank(fitness_values)
        selection_probabilities = selection_probabilities_rank(ranks)
        parent_indices = np.random.choice(np.arange(population_size), size=2, replace=False, p=selection_probabilities)
        parents = select_parents_rank(population, selection_probabilities, 2*(population_size-elite_size))

        # Crossover parents to create offspring
        offspring = []
        for j in range(0, len(parents), 2):
            offspring.extend(crossover(parents[j], parents[j+1], crossover_rate))

        # Mutate offspring
        mutated_offspring = [mutate(individual, mutation_rate) for individual in offspring]

        # Add elite to population
        new_population = elite + mutated_offspring

        # Keep population size constant
        new_population = new_population[:population_size]

        # Update population for next generation
        population = new_population

    # Return the best individual found during the entire evolution process

    best_individual = max(population, key=fitness)
    curr_best_fitness = fitness(best_individual)
    if curr_best_fitness > best_fitness:
        best_individual_ = best_individual
        best_fitness = curr_best_fitness
    print("curr_best: ", curr_best_fitness, "  best_so_far: ", best_fitness)
    print(best_individual)
    print(best_individual_)
    print()
    return best_individual_, best_fitness

In [4]:
################# IRIS DATASET ################
# from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# iris = load_iris()
# x = iris.data
# y = iris.target
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# dt = DecisionTree()
# tree = dt.fit(X_train, y_train)
# y_pred = [dt.predict(X, tree) for X in X_test]
# accuracy = sum(y_pred == y_test) / len(y_test)
# print(f"Accuracy: {accuracy}")
# print(y_test)
# print(y_pred)

################ WINE DATASET ##################
from sklearn.datasets import load_wine
wine = load_wine()
x = wine.data
y = wine.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

dt = DecisionTree()
tree = dt.fit(X_train, y_train)
y_pred = [dt.predict(X, tree) for X in X_test]
accuracy = sum(y_pred == y_test) / len(y_test)
print(y_test)
print(y_pred)
print(f"Accuracy with Scikit Learn Decision Tree Classifier: {accuracy}")
print()
print()



# Find the best individual and print the decision tree
print('=' * 100)
print("Optimization Starts using Genetic Algorithm")
print('=' * 100)
best_individual, best_fitness = genetic_algorithm(100, 15, 0.1, 0.1)
print(best_individual)
print("Best Accuracy after Optimization using Genetic Algorith: ", best_fitness)

[0 0 2 0 1 0 1 2 1 2 0 2 0 1 0 1 1 1 0 1 0 1 1 2 2 2 1 1 1 0 0 1 2 0 0 0]
[0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0]
Accuracy with Scikit Learn Decision Tree Classifier: 0.9166666666666666


Optimization Starts using Genetic Algorithm
Iteration  0
curr_best:  0.75   best_so_far:  0.75
[19, 0.06183600883072026, 0.07603987139141197, 1.0]
[19, 0.06183600883072026, 0.07603987139141197, 1.0]


Iteration  1
curr_best:  0.7777777777777778   best_so_far:  0.7777777777777778
[8, 0.05883271128011678, 0.01819514608685875, 1.0]
[8, 0.05883271128011678, 0.01819514608685875, 1.0]


Iteration  2
curr_best:  0.8888888888888888   best_so_far:  0.8888888888888888
[9, 0.9409255229019605, 0.07092808706355679, 1.0]
[9, 0.9409255229019605, 0.07092808706355679, 1.0]


Iteration  3
curr_best:  0.8888888888888888   best_so_far:  0.8888888888888888
[8, 0.08428484487106422, 0.01819514608685875, 1.0]
[9, 0.9409255229019605, 0.07092808706355679, 1.0